In [2]:
# 连接colab服务器，gdrive云盘
import os
from google.colab import drive
drive.mount('/gdrive')
os.symlink('/gdrive/My Drive', '/content/gdrive')
!ls -l /content/gdrive/

Mounted at /gdrive
total 16
drwx------ 2 root root 4096 Feb 24 11:59 EFNET
drwx------ 2 root root 4096 Jan 31  2020 google_AI
drwx------ 2 root root 4096 Feb 21 14:16 HRNET
drwx------ 2 root root 4096 Feb 23 14:00 RESNET


In [3]:
# 安装必要包
!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install efficientnet

/bin/bash: google-drive-ocamlfuse: command not found
     |████████████████████████████████| 51kB 5.0MB/s 


In [4]:
# 设置工作目录
import os 
os.chdir('gdrive/EFNET')
!ls

15model.h5  6model.h5	 EFNET.py     model.h5	   record2
18model.h5  EFNET.ipynb  GET_DATA.py  __pycache__


In [ ]:
# 训练代码
from EFNET import UEfficientNet
from GET_DATA import val_record, train_record, tfrecords_reader_dataset
import tensorflow.keras as keras
from tensorflow.keras import backend as K
import tensorflow as tf
import numpy as np
from keras import callbacks

val_record='record2/valid_128.record'
train_record='record2/train_128.record'

print(val_record)
print(train_record)

batch_size = 8
epoch=1000
class_weight=[1,1,4,4,4,4,4,4,2,6]
dice_weight = np.array([0.03,0.03,0.12,0.12,0.12,0.12,0.12,0.12,0.06,0.18], dtype="float32")
dice_weight = tf.convert_to_tensor(dice_weight, dtype="float32")

# 模拟退火学习率
class CosineAnnealing(callbacks.Callback):
    """Cosine annealing according to DECOUPLED WEIGHT DECAY REGULARIZATION.

    # Arguments
        eta_max: float, eta_max in eq(5).
        eta_min: float, eta_min in eq(5).
        total_iteration: int, Ti in eq(5).
        iteration: int, T_cur in eq(5).
        verbose: 0 or 1.
    """

    def __init__(self, eta_max=1, eta_min=0., total_iteration=0, iteration=3, verbose=1, **kwargs):
        super(CosineAnnealing, self).__init__()

        global lr_list

        lr_list = []
        self.eta_max = eta_max
        self.eta_min = eta_min
        self.verbose = verbose
        self.total_iteration = total_iteration
        self.iteration = iteration

    def on_train_begin(self, logs=None):
        self.lr = K.get_value(self.model.optimizer.lr)

    def on_train_end(self, logs=None):
        K.set_value(self.model.optimizer.lr, self.lr)

    def on_batch_end(self, epoch, logs=None):
        self.iteration += 1
        eta_t = self.eta_min + (self.eta_max - self.eta_min) * 0.5 * (
                1 + np.cos(np.pi * self.iteration / self.total_iteration))
        new_lr=K.clip(self.lr * eta_t,0.00001,0.0001)
        K.set_value(self.model.optimizer.lr, new_lr)

# 交并比损失
def dice_loss(y_true,y_pred, smooth=1):
    y_true=y_true[:,:,:,0]
    y_true=K.expand_dims(y_true,axis=-1)
    y_true=K.squeeze(K.one_hot(K.cast(y_true,'int32'),num_classes=10),axis=3)
    y_pred=K.one_hot(K.argmax(y_pred),num_classes=10)
    intersection = K.sum(K.abs(y_true*y_pred), axis=(0, 1, 2))
    result = (intersection) / (K.sum(y_true, (0, 1, 2)) + K.sum(y_pred, (0, 1, 2)) + 1e-6-intersection)
    all_num = K.sum(K.abs(y_true), axis=(0, 1, 2))
    all_ = len(tf.where((tf.equal(all_num,0.))))
    all_=K.cast(all_, 'float32')
    return (K.sum((1-result))-all_)/(10-all_)
    return K.sum((1-result)/10)

# 带权重交叉熵损失函数
def weighted_loss(y_true,y_pred):
    w=y_true[:,:,:,1]
    y_true = y_true[:, :, :, 0]
    y_true = K.expand_dims(y_true, axis=-1)
    w_loss=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_pred,labels=tf.squeeze(y_true,axis=3))
    out=w_loss*K.cast(w,'float32')
    return K.mean(out)

# 交并比正确率
def accu(y_true,y_pred):
    y_true = y_true[:, :, :, 0]
    y_true = K.expand_dims(y_true, axis=-1)
    y_true=K.squeeze(K.one_hot(K.cast(y_true,'int32'),num_classes=10),axis=3)
    y_pred=K.one_hot(K.argmax(y_pred),num_classes=10)
    intersection = K.sum(K.abs(y_true*y_pred), axis=(0, 1, 2))
    iousum=K.sum(y_true, (0, 1, 2))
    return K.mean(intersection/(iousum+1e-6))

def my_loss(y_true,y_pred):
    return dice_loss(y_true,y_pred, smooth=1)+weighted_loss(y_true,y_pred)

def acc_me(y_true,y_pred):
    return 1-dice_loss(y_true,y_pred)


# 搭建模型
model = UEfficientNet()
# model.load_weights('15model.h5')
model.compile(optimizer=keras.optimizers.Adam(),loss=my_loss, metrics=[acc_me,accu])
model.summary()
# train_data = tfrecords_reader_dataset(train_record, batch_size=batch_size)
valid_data = tfrecords_reader_dataset(val_record, batch_size)
train_data =tfrecords_reader_dataset(train_record, batch_size=batch_size)
epochs = [3*i for i in range(1,30)]

# 16008为总样本数量 0.8为训练样本比重

for Ti in epochs:
    file_path=str(Ti)+'model.h5'
    model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='val_acc_me', verbose=1,save_best_only=True, save_weights_only=True, period=1)
    reduce_lr = CosineAnnealing(eta_max=1, eta_min=0.00001, total_iteration=Ti * (int(16008*0.8) // batch_size), iteration=3, verbose=1)
    model.fit_generator(train_data, steps_per_epoch=int(16008 * 0.8 / batch_size), epochs=Ti,
                    callbacks=[model_checkpoint, reduce_lr], shuffle=False,
                    validation_data=valid_data, validation_steps=int(16008 * 0.2 / batch_size))


record2/valid_128.record
record2/train_128.record
258441216/258434480 [==============================] - 4s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 128, 128, 64) 1728        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 128, 128, 64) 256         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 128, 128, 64) 0           stem_bn[0][

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/18
 976/1600 [=================>............] - ETA: 8:17 - loss: 0.7068 - acc_me: 0.5579 - accu: 0.4632